In [1]:
import sys,  os
sys.path.insert(0, '../exostriker/lib')
#sys.path.append('../lib/') #RV_mod directory must be in your path
#sys.path.append('../lib/RV_mod/') #RV_mod directory must be in your path
import RV_mod as rv
import numpy as np
#import glob
import dill
#import os
#from subprocess import PIPE, Popen 
#import time 
%matplotlib

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable

import pathos as ps
#import gc

import itertools

def closest(lst, value):
    closest_value = lst[min(range(len(lst)), key = lambda i: abs(lst[i]-value))]
    indx = int(np.where(lst==closest_value)[0])
    return closest_value, indx


Using matplotlib backend: <object object at 0x7f9918946b90>


In [2]:
mpl.rc('text',usetex=True)
font = {'family' : 'normal','weight' : 'bold','size'   : 16,'serif':['Helvetica']}
mpl.rc('font', **font)
format_im = 'pdf'
dpi = 300

In [3]:
def run_detection_rate(obj2,options):

    from tqdm import tqdm

    results = {"P":[],"K":[],"M0":[],'gls':[],'power':[],'peaks':[],'input':options}
    
 
    for P in tqdm(options['P']): 
        for K in tqdm(options['K']): 
            for M0 in tqdm(options['M0']): 

                #obj2 = dill.copy(inject_signal(obj, K=K,P=P,M0=M0))

                sinewave = K * np.sin(2 * np.pi * 1/P * obj2.fit_results.rv_model.jd + np.radians(M0))


                
                if options['gls_o-c']:
                    new_RVs = obj2.fit_results.rv_model.o_c + sinewave
                else:
                    new_RVs = obj2.fit_results.rv_model.rvs + sinewave                   

                RV_per = rv.gls.Gls((obj2.fit_results.rv_model.jd, new_RVs, obj2.fit_results.rv_model.rv_err), fast=True, verbose=False, norm="ZK",ofac=options['gls_ofac'], fbeg=1/options['gls_Pmax'], fend=1/options['gls_Pmin']) 


                text_peaks, pos_peaks = rv.identify_power_peaks(1/RV_per.freq, RV_per.power, power_level = options['power_levels'], sig_level = RV_per.powerLevel(np.array(options['power_levels'])) )

                results["P"].append(P)
                results["K"].append(K)                
                results["M0"].append(M0)
                results["gls"].append(dill.copy(RV_per.powerLevel(np.array(options['power_levels']))))
                results["peaks"].append(dill.copy(pos_peaks[0][0:10]))
                results["power"].append(dill.copy(pos_peaks[1][0:10]))

    return results

In [4]:
file_pi = open(r"/Users/antoniagraefinstauffenberg//Desktop/GJ 581/My sessions/HARPS/HARPS_Bin05_2pl.ses", 'rb')
fit = dill.load(file_pi)
file_pi.close()   

In [5]:
########### Define the ranges and steps of P,K, and M0 to be explored
P  = np.arange(60,70,0.5)
K  = np.arange(0.2,5,0.2)
M0 = np.arange(0,360,36)
########### GLS FAPs ###################
power_levels = np.array([0.1,0.01,0.01])
########### input for the script #######
input_opt = {'P':P, 'K':K, 'M0':M0,'gls_ofac':5, 'gls_Pmax':200,'gls_Pmin':2.0,'gls_o-c':True,'power_levels':power_levels}

det_rate_results = rv.run_detection_rate(fit,input_opt)



  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:00<00:02,  3.44it/s]

 20%|██        | 2/10 [00:00<00:02,  3.63it/s]

 30%|███       | 3/10 [00:00<00:01,  3.68it/s]

 40%|████      | 4/10 [00:01<00:01,  3.71it/s]

 50%|█████     | 5/10 [00:01<00:01,  3.73it/s]

 60%|██████    | 6/10 [00:01<00:01,  3.74it/s]

 70%|███████   | 7/10 [00:01<00:00,  3.75it/s]

 80%|████████  | 8/10 [00:02<00:00,  3.76it/s]

 90%|█████████ | 9/10 [00:02<00:00,  3.76it/s]

100%|██████████| 10/10 [00:02<00:00,  3.72it/s]

  4%|▍         | 1/24 [00:02<01:01,  2.69s/it]

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:00<00:02,  3.63it/s]

 20%|██        | 2/10 [00:00<00:02,  3.72it/s]

 30%|███       | 3/10 [00:00<00:01,  3.68it/s]

 40%|████      | 4/10 [00:01<00:01,  3.70it/s]

 50%|█████     | 5/10 [00:01<00:01,  3.65it/s]

 60%|██████    | 6/10 [00:01<00:01,  3.49it/s]

 70%|███████   | 7/10 [00:01<00:00,  3.55it/s]

 80%|████████  

 80%|████████  | 8/10 [00:02<00:00,  3.82it/s]

 90%|█████████ | 9/10 [00:02<00:00,  3.82it/s]

100%|██████████| 10/10 [00:02<00:00,  3.81it/s]

 54%|█████▍    | 13/24 [00:35<00:29,  2.68s/it]

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:00<00:02,  3.75it/s]

 20%|██        | 2/10 [00:00<00:02,  3.81it/s]

 30%|███       | 3/10 [00:00<00:01,  3.82it/s]

 40%|████      | 4/10 [00:01<00:01,  3.81it/s]

 50%|█████     | 5/10 [00:01<00:01,  3.82it/s]

 60%|██████    | 6/10 [00:01<00:01,  3.81it/s]

 70%|███████   | 7/10 [00:01<00:00,  3.81it/s]

 80%|████████  | 8/10 [00:02<00:00,  3.82it/s]

 90%|█████████ | 9/10 [00:02<00:00,  3.82it/s]

100%|██████████| 10/10 [00:02<00:00,  3.81it/s]

 58%|█████▊    | 14/24 [00:38<00:26,  2.67s/it]

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:00<00:02,  3.77it/s]

 20%|██        | 2/10 [00:00<00:02,  3.80it/s]

 30%|███       | 3/10 [00:00<00:01,  3.81it/s]

 40%|████      | 4/10 [00:01<00:01,  3.79it/s]

 50%

 40%|████      | 4/10 [00:01<00:01,  3.80it/s]

 50%|█████     | 5/10 [00:01<00:01,  3.81it/s]

 60%|██████    | 6/10 [00:01<00:01,  3.81it/s]

 70%|███████   | 7/10 [00:01<00:00,  3.81it/s]

 80%|████████  | 8/10 [00:02<00:00,  3.81it/s]

 90%|█████████ | 9/10 [00:02<00:00,  3.81it/s]

100%|██████████| 10/10 [00:02<00:00,  3.80it/s]

  8%|▊         | 2/24 [00:05<00:58,  2.66s/it]

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:00<00:02,  3.65it/s]

 20%|██        | 2/10 [00:00<00:02,  3.51it/s]

 30%|███       | 3/10 [00:00<00:01,  3.64it/s]

 40%|████      | 4/10 [00:01<00:01,  3.71it/s]

 50%|█████     | 5/10 [00:01<00:01,  3.75it/s]

 60%|██████    | 6/10 [00:01<00:01,  3.79it/s]

 70%|███████   | 7/10 [00:01<00:00,  3.80it/s]

 80%|████████  | 8/10 [00:02<00:00,  3.81it/s]

 90%|█████████ | 9/10 [00:02<00:00,  3.81it/s]

100%|██████████| 10/10 [00:02<00:00,  3.76it/s]

 12%|█▎        | 3/24 [00:07<00:55,  2.66s/it]

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█

 20%|██        | 2/10 [00:00<00:02,  3.77it/s]

 30%|███       | 3/10 [00:00<00:01,  3.80it/s]

 40%|████      | 4/10 [00:01<00:01,  3.81it/s]

 50%|█████     | 5/10 [00:01<00:01,  3.81it/s]

 60%|██████    | 6/10 [00:01<00:01,  3.82it/s]

 70%|███████   | 7/10 [00:01<00:00,  3.82it/s]

 80%|████████  | 8/10 [00:02<00:00,  3.82it/s]

 90%|█████████ | 9/10 [00:02<00:00,  3.82it/s]

100%|██████████| 10/10 [00:02<00:00,  3.80it/s]

 62%|██████▎   | 15/24 [00:39<00:23,  2.63s/it]

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:00<00:02,  3.71it/s]

 20%|██        | 2/10 [00:00<00:02,  3.77it/s]

 30%|███       | 3/10 [00:00<00:01,  3.79it/s]

 40%|████      | 4/10 [00:01<00:01,  3.78it/s]

 50%|█████     | 5/10 [00:01<00:01,  3.79it/s]

 60%|██████    | 6/10 [00:01<00:01,  3.81it/s]

 70%|███████   | 7/10 [00:01<00:00,  3.81it/s]

 80%|████████  | 8/10 [00:02<00:00,  3.81it/s]

 90%|█████████ | 9/10 [00:02<00:00,  3.81it/s]

100%|██████████| 10/10 [00:02<00:00,  3.80it/s

100%|██████████| 10/10 [00:02<00:00,  3.83it/s]

 12%|█▎        | 3/24 [00:07<00:54,  2.62s/it]

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:00<00:02,  3.79it/s]

 20%|██        | 2/10 [00:00<00:02,  3.83it/s]

 30%|███       | 3/10 [00:00<00:01,  3.83it/s]

 40%|████      | 4/10 [00:01<00:01,  3.83it/s]

 50%|█████     | 5/10 [00:01<00:01,  3.83it/s]

 60%|██████    | 6/10 [00:01<00:01,  3.83it/s]

 70%|███████   | 7/10 [00:01<00:00,  3.83it/s]

 80%|████████  | 8/10 [00:02<00:00,  3.81it/s]

 90%|█████████ | 9/10 [00:02<00:00,  3.81it/s]

100%|██████████| 10/10 [00:02<00:00,  3.82it/s]

 17%|█▋        | 4/24 [00:10<00:52,  2.62s/it]

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:00<00:02,  3.80it/s]

 20%|██        | 2/10 [00:00<00:02,  3.82it/s]

 30%|███       | 3/10 [00:00<00:01,  3.82it/s]

 40%|████      | 4/10 [00:01<00:01,  3.83it/s]

 50%|█████     | 5/10 [00:01<00:01,  3.84it/s]

 60%|██████    | 6/10 [00:01<00:01,  3.84it/s]

 70%|█

...

 80%|████████  | 8/10 [00:01<00:00,  6.03it/s]

 90%|█████████ | 9/10 [00:01<00:00,  6.04it/s]

100%|██████████| 10/10 [00:01<00:00,  6.04it/s]

 50%|█████     | 12/24 [00:19<00:19,  1.65s/it]

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:00<00:01,  6.01it/s]

 20%|██        | 2/10 [00:00<00:01,  6.04it/s]

 30%|███       | 3/10 [00:00<00:01,  6.07it/s]

 40%|████      | 4/10 [00:00<00:00,  6.09it/s]

 50%|█████     | 5/10 [00:00<00:00,  6.09it/s]

 60%|██████    | 6/10 [00:00<00:00,  6.10it/s]

 70%|███████   | 7/10 [00:01<00:00,  6.09it/s]

 80%|████████  | 8/10 [00:01<00:00,  6.10it/s]

 90%|█████████ | 9/10 [00:01<00:00,  6.10it/s]

100%|██████████| 10/10 [00:01<00:00,  6.09it/s]

 54%|█████▍    | 13/24 [00:21<00:18,  1.65s/it]

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:00<00:01,  5.99it/s]

 20%|██        | 2/10 [00:00<00:01,  6.05it/s]

 30%|███       | 3/10 [00:00<00:01,  6.07it/s]

 40%|████      | 4/10 [00:00<00:00,  6.08it/s]

 50%

 40%|████      | 4/10 [00:00<00:00,  6.05it/s]

 50%|█████     | 5/10 [00:00<00:00,  6.07it/s]

 60%|██████    | 6/10 [00:00<00:00,  6.04it/s]

 70%|███████   | 7/10 [00:01<00:00,  6.05it/s]

 80%|████████  | 8/10 [00:01<00:00,  6.06it/s]

 90%|█████████ | 9/10 [00:01<00:00,  6.07it/s]

100%|██████████| 10/10 [00:01<00:00,  6.05it/s]

  4%|▍         | 1/24 [00:01<00:38,  1.65s/it]

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:00<00:01,  6.01it/s]

 20%|██        | 2/10 [00:00<00:01,  6.05it/s]

 30%|███       | 3/10 [00:00<00:01,  6.07it/s]

 40%|████      | 4/10 [00:00<00:00,  6.09it/s]

 50%|█████     | 5/10 [00:00<00:00,  6.09it/s]

 60%|██████    | 6/10 [00:00<00:00,  6.09it/s]

 70%|███████   | 7/10 [00:01<00:00,  6.09it/s]

 80%|████████  | 8/10 [00:01<00:00,  6.09it/s]

 90%|█████████ | 9/10 [00:01<00:00,  6.09it/s]

100%|██████████| 10/10 [00:01<00:00,  6.08it/s]

  8%|▊         | 2/24 [00:03<00:36,  1.65s/it]

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█

 20%|██        | 2/10 [00:00<00:01,  6.14it/s]

 30%|███       | 3/10 [00:00<00:01,  6.17it/s]

 40%|████      | 4/10 [00:00<00:00,  6.16it/s]

 50%|█████     | 5/10 [00:00<00:00,  6.18it/s]

 60%|██████    | 6/10 [00:00<00:00,  6.18it/s]

 70%|███████   | 7/10 [00:01<00:00,  6.20it/s]

 80%|████████  | 8/10 [00:01<00:00,  6.18it/s]

 90%|█████████ | 9/10 [00:01<00:00,  6.18it/s]

100%|██████████| 10/10 [00:01<00:00,  6.18it/s]

 58%|█████▊    | 14/24 [00:23<00:16,  1.64s/it]

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:00<00:01,  6.09it/s]

 20%|██        | 2/10 [00:00<00:01,  6.14it/s]

 30%|███       | 3/10 [00:00<00:01,  6.17it/s]

 40%|████      | 4/10 [00:00<00:00,  6.18it/s]

 50%|█████     | 5/10 [00:00<00:00,  6.18it/s]

 60%|██████    | 6/10 [00:00<00:00,  6.19it/s]

 70%|███████   | 7/10 [00:01<00:00,  6.19it/s]

 80%|████████  | 8/10 [00:01<00:00,  6.19it/s]

 90%|█████████ | 9/10 [00:01<00:00,  6.19it/s]

100%|██████████| 10/10 [00:01<00:00,  6.18it/s

100%|██████████| 10/10 [00:01<00:00,  6.18it/s]

  8%|▊         | 2/24 [00:03<00:35,  1.62s/it]

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:00<00:01,  6.12it/s]

 20%|██        | 2/10 [00:00<00:01,  6.14it/s]

 30%|███       | 3/10 [00:00<00:01,  6.17it/s]

 40%|████      | 4/10 [00:00<00:00,  6.17it/s]

 50%|█████     | 5/10 [00:00<00:00,  6.18it/s]

 60%|██████    | 6/10 [00:00<00:00,  6.17it/s]

 70%|███████   | 7/10 [00:01<00:00,  6.18it/s]

 80%|████████  | 8/10 [00:01<00:00,  6.18it/s]

 90%|█████████ | 9/10 [00:01<00:00,  6.18it/s]

100%|██████████| 10/10 [00:01<00:00,  6.18it/s]

 12%|█▎        | 3/24 [00:04<00:34,  1.62s/it]

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:00<00:01,  6.13it/s]

 20%|██        | 2/10 [00:00<00:01,  6.17it/s]

 30%|███       | 3/10 [00:00<00:01,  6.17it/s]

 40%|████      | 4/10 [00:00<00:00,  6.16it/s]

 50%|█████     | 5/10 [00:00<00:00,  6.17it/s]

 60%|██████    | 6/10 [00:00<00:00,  6.19it/s]

 70%|█

 80%|████████  | 8/10 [00:01<00:00,  6.18it/s]

 90%|█████████ | 9/10 [00:01<00:00,  6.19it/s]

100%|██████████| 10/10 [00:01<00:00,  6.18it/s]

 62%|██████▎   | 15/24 [00:24<00:14,  1.62s/it]

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:00<00:01,  6.09it/s]

 20%|██        | 2/10 [00:00<00:01,  6.16it/s]

 30%|███       | 3/10 [00:00<00:01,  6.17it/s]

 40%|████      | 4/10 [00:00<00:00,  6.17it/s]

 50%|█████     | 5/10 [00:00<00:00,  6.19it/s]

 60%|██████    | 6/10 [00:00<00:00,  6.19it/s]

 70%|███████   | 7/10 [00:01<00:00,  6.20it/s]

 80%|████████  | 8/10 [00:01<00:00,  6.20it/s]

 90%|█████████ | 9/10 [00:01<00:00,  6.20it/s]

100%|██████████| 10/10 [00:01<00:00,  6.18it/s]

 67%|██████▋   | 16/24 [00:25<00:12,  1.62s/it]

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:00<00:01,  6.11it/s]

 20%|██        | 2/10 [00:00<00:01,  6.14it/s]

 30%|███       | 3/10 [00:00<00:01,  6.16it/s]

 40%|████      | 4/10 [00:00<00:00,  6.17it/s]

 50%

In [12]:
f = open("Extracted_det_rate_results.pkl","wb")
dill.dump(det_rate_results,f)
f.close()

In [6]:
########## Define the 2d array needed for imshow()

Prob_array = np.zeros((len(K),len(P)))

########### Loop over all P,K and M0 and fill the 2d array

for i,iP in enumerate(P):
    for j,jK in enumerate(K):
        prob = 0
       
        M0_ind = np.where(np.logical_and(det_rate_results["P"] == iP, det_rate_results["K"] == jK))[0]
        for z, indM0 in enumerate(M0_ind):

            Close_Peak,indx = closest(det_rate_results["peaks"][indM0],iP)
            
            if abs(Close_Peak - iP) < iP*0.05 and det_rate_results["power"][indM0][indx] >= det_rate_results["gls"][indM0][2]:
                #print(det_rate_results["power"][indM0][indx],det_rate_results["gls"][indM0][2])
                prob= prob + 1
                #print(iP,Close_Peak)
        Prob_array[j,i] = (prob/len(M0_ind))
        

   
############# Plotting ###############

planet_p = [3.153, 5.368, 12.919, 66.688]
planet_p_err = [0.001, 0.000, 0.001, 0.020]

planet_K = [1.782, 12.488, 3.095, 1.618]
planet_K_err = [0.384, 0.142, 0.063, 0.195]
 
#fig = plt.figure(0)
#ax1 = plt.subplot(1, 1, 1)

extent=[min(P), max(P)+1,min(K),max(K)+1]

plt.imshow(Prob_array, cmap = 'magma', interpolation='nearest',origin='lower',aspect='auto',extent=extent)
plt.errorbar(66.688, 1.618, xerr = 0.020, yerr = 0.195, capsize=2, fmt = "o", color = "limegreen")
plt.title("HARPS")
plt.ylabel('K [m/s]',fontsize=18,labelpad= 10, rotation = 'vertical')
plt.xlabel('P [days]',fontsize=18)
#divider = make_axes_locatable(ax1)
#cax = divider.append_axes("right", size="5%", pad=0.05)

#col1 = plt.colorbar(im, cax=cax, orientation="vertical")
plt.colorbar().set_label('Detection probability', rotation = 90, labelpad=12)

plt.tight_layout(pad=0.4, w_pad=0.3, h_pad=1.0)

plt.savefig('/Users/antoniagraefinstauffenberg//Desktop/GJ 581/Results/ImshowPlot_HARPS1.pdf', format="pdf",dpi=dpi, bbox_inches='tight')

plt.show()